In [3]:
import folium
import geopandas as gpd
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Fonction pour ouvrir une fenêtre de sélection de fichier
def select_file():
    root = Tk()
    root.withdraw()  # Masquer la fenêtre principale de Tkinter
    file_path = askopenfilename(
        filetypes=[("GeoJSON files", "*.json"), ("All files", "*.*")]
    )
    root.destroy()
    return file_path

# Ouvrir la fenêtre de sélection de fichier
geojson_path = select_file()

if geojson_path:
    # Charger les données GeoJSON
    geo_data = gpd.read_file(geojson_path)

    # Créer une carte folium
    m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

    # Couleurs associées aux partis
    party_colors = {
        "Non défini": "black",
        "PS - Front populaire": "pink",
        "LFI - Front populaire": "red",
        "EELV - Front populaire": "green",
        "PCF - Front populaire": "darkred",
        "Autres - Front populaire": "darkred",
        "Modem - Ensemble": "yellow",
        "Renaissance - Ensemble": "gold",
        "Autre - Ensemble": "gold",
        "LR": "blue",
        "LR - RN": "darkblue",
        "RN": "navy",
        "Reconquête": "brown",
        "Independant": "grey"
    }

    # Initialiser le compteur de circonscriptions par parti
    party_counts = {party: 0 for party in party_colors.keys()}
    party_counts["Non défini"] = len(geo_data)  # Toutes les circonscriptions commencent comme "Non défini"

    # Fonction pour afficher un popup et permettre de choisir un parti
    def style_function(feature, party="Non défini"):
        color = party_colors.get(party, "black")
        return {
            'fillOpacity': 0.5 if color != "black" else 0.1,
            'weight': 1,
            'color': color
        }

    # Fonction pour générer le contenu du popup
    def generate_popup(feature, geo_json):
        options = "".join([f'<option value="{party}">{party}</option>' for party in party_colors.keys()])
        popup_content = f"""
        <h4>Circonscription {feature['num_circ']} - {feature['nom_dpt']}</h4>
        <form action="javascript:void(0);">
            <label for="party">Choisir un parti :</label><br>
            <select id="party_{feature['ID']}" name="party" onchange="updateParty('{feature['ID']}')">
                {options}
            </select>
            <input type="hidden" id="geo_json_{feature['ID']}" value="{geo_json.get_name()}">
            <input type="hidden" id="initial_party_{feature['ID']}" value="Non défini">
        </form>
        """
        return popup_content

    # Fonction pour ajouter un GeoJson avec popup à la carte
    def add_geojson_with_popup(geo_data, map_object):
        for _, row in geo_data.iterrows():
            geo_json = folium.GeoJson(
                row['geometry'],
                style_function=lambda feature: style_function(feature, "Non défini")
            )
            popup = folium.Popup(generate_popup(row, geo_json), max_width=300)
            popup.add_to(geo_json)
            geo_json.add_to(map_object)

    # Ajouter les circonscriptions à la carte avec un popup interactif
    add_geojson_with_popup(geo_data, m)

    # Ajouter un script JavaScript pour mettre à jour la couleur en fonction du parti choisi
    script = f"""
    <script>
    var partyCounts = {party_counts};

    function updateParty(circ_id) {{
        var select = document.getElementById('party_' + circ_id);
        var party = select.value;
        var geoJsonId = document.getElementById('geo_json_' + circ_id).value;
        var geoJsonLayer = window[geoJsonId];

        var initialPartyElement = document.getElementById('initial_party_' + circ_id);
        var initialParty = initialPartyElement.value;

        if (initialParty !== "Non défini") {{
            partyCounts[initialParty]--;
            document.getElementById(initialParty + '_count').innerText = partyCounts[initialParty];
        }} else {{
            partyCounts["Non défini"]--;
            document.getElementById("Non défini_count").innerText = partyCounts["Non défini"];
        }}

        geoJsonLayer.setStyle({{
            fillColor: {{
                "Non défini": "black",
                "PS - Front populaire": "pink",
                "LFI - Front populaire": "red",
                "EELV - Front populaire": "green",
                "PCF - Front populaire": "darkred",
                "Autres - Front populaire": "darkred",
                "Modem - Ensemble": "yellow",
                "Renaissance - Ensemble": "gold",
                "Autre - Ensemble": "gold",
                "LR": "blue",
                "LR - RN": "darkblue",
                "RN": "navy",
                "Reconquête": "brown",
                "Independant": "grey"
            }}[party],
            fillOpacity: party === "Non défini" ? 0.1 : 0.5,
        }});

        if (party !== "Non défini") {{
            partyCounts[party]++;
            document.getElementById(party + '_count').innerText = partyCounts[party];
        }}

        initialPartyElement.value = party;
    }}

    function updateForeignParty(circ_id) {{
        var select = document.getElementById('foreign_party_' + circ_id);
        var party = select.value;
        var initialPartyElement = document.getElementById('foreign_initial_party_' + circ_id);
        var initialParty = initialPartyElement.value;

        if (initialParty !== "Non défini") {{
            partyCounts[initialParty]--;
            document.getElementById(initialParty + '_count').innerText = partyCounts[initialParty];
        }} else {{
            partyCounts["Non défini"]--;
            document.getElementById("Non défini_count").innerText = partyCounts["Non défini"];
        }}

        if (party !== "Non défini") {{
            partyCounts[party]++;
            document.getElementById(party + '_count').innerText = partyCounts[party];
        }}

        initialPartyElement.value = party;
    }}

    function resetMap() {{
        var geoJsonLayers = document.getElementsByClassName('leaflet-interactive');
        for (var i = 0; i < geoJsonLayers.length; i++) {{
            geoJsonLayers[i].setAttribute('style', 'fill: black; fill-opacity: 0.1; stroke: black;');
        }}

        for (var party in partyCounts) {{
            partyCounts[party] = 0;
            document.getElementById(party + '_count').innerText = partyCounts[party];
        }}

        partyCounts["Non défini"] = {len(geo_data)};
        document.getElementById("Non défini_count").innerText = partyCounts["Non défini"];

        var foreignSelects = document.getElementsByClassName('foreign-select');
        for (var i = 0; i < foreignSelects.length; i++) {{
            foreignSelects[i].value = "Non défini";
            document.getElementById('foreign_initial_party_' + foreignSelects[i].id.split('_')[2]).value = "Non défini";
        }}
    }}

    function toggleForeignPanel() {{
        var panel = document.getElementById('foreign-panel');
        if (panel.style.display === 'none' || panel.style.display === '') {{
            panel.style.display = 'block';
        }} else {{
            panel.style.display = 'none';
        }}
    }}

    function toggleCounterPanel() {{
        var panel = document.getElementById('counter-panel');
        var button = document.getElementById('toggle-counter-panel');
        if (panel.style.display === 'none' || panel.style.display === '') {{
            panel.style.display = 'block';
            button.innerText = 'Réduire le compteur de circonscriptions';
        }} else {{
            panel.style.display = 'none';
            button.innerText = 'Afficher le compteur de circonscriptions';
        }}
    }}
    </script>
    """
    m.get_root().html.add_child(folium.Element(script))

    # Ajouter les compteurs de parti, le bouton de réinitialisation et l'indicateur de majorité
    counter_html = f"""
    <button id="toggle-counter-panel" style="position: fixed; bottom: 60px; left: 10px; z-index: 1000;" onclick="toggleCounterPanel()">Afficher le compteur de circonscriptions</button>
    <div id="counter-panel" style="position: fixed; bottom: 100px; left: 10px; display: none; background: white; padding: 10px; border: 1px solid black; z-index: 1000;">
        <h4>Compteur de circonscriptions par parti</h4>
                <p>Majorité: 289</p>
        <p>Non défini: <span id="Non défini_count">{len(geo_data)}</span></p>
        <p>PS - Front populaire: <span id="PS - Front populaire_count">0</span></p>
        <p>LFI - Front populaire: <span id="LFI - Front populaire_count">0</span></p>
        <p>EELV - Front populaire: <span id="EELV - Front populaire_count">0</span></p>
        <p>PCF - Front populaire: <span id="PCF - Front populaire_count">0</span></p>
        <p>Autres - Front populaire: <span id="Autres - Front populaire_count">0</span></p>
        <p>Modem - Ensemble: <span id="Modem - Ensemble_count">0</span></p>
        <p>Renaissance - Ensemble: <span id="Renaissance - Ensemble_count">0</span></p>
        <p>Autre - Ensemble: <span id="Autre - Ensemble_count">0</span></p>
        <p>LR: <span id="LR_count">0</span></p>
        <p>LR - RN: <span id="LR - RN_count">0</span></p>
        <p>RN: <span id="RN_count">0</span></p>
        <p>Reconquête: <span id="Reconquête_count">0</span></p>
        <p>Independant: <span id="Independant_count">0</span></p>
        <button onclick="resetMap()">Réinitialiser la carte</button>
    </div>
    """

    m.get_root().html.add_child(folium.Element(counter_html))

    # Ajouter les choix pour les circonscriptions des Français de l'étranger dans un panneau repliable
    foreign_circonscriptions = [
        ("Première circonscription", "Les États-Unis et le Canada"),
        ("Deuxième circonscription", "Le Mexique, l'Amérique centrale, les Caraïbes et l'Amérique du Sud"),
        ("Troisième circonscription", "L'Europe du Nord"),
        ("Quatrième circonscription", "Le Benelux"),
        ("Cinquième circonscription", "La péninsule Ibérique"),
        ("Sixième circonscription", "La Suisse et le Liechtenstein"),
        ("Septième circonscription", "L'Europe centrale et les Balkans"),
        ("Huitième circonscription", "Chypre, Grèce, Israël, Italie, Malte, Turquie et Territoires palestiniens"),
        ("Neuvième circonscription", "Le Maghreb et l'Afrique de l'Ouest"),
        ("Dixième circonscription", "Le Proche-Orient et la majeure partie de l'Afrique"),
        ("Onzième circonscription", "La Biélorussie, la Moldavie, l'Ukraine, la Russie, les pays du Caucase, l'Asie et l'Océanie")
    ]

    foreign_html = """
    <button id="toggle-foreign-panel" style="position: fixed; bottom: 10px; right: 10px; z-index: 1000;" onclick="toggleForeignPanel()">Afficher les résultats des Français de l'étranger</button>
    <div id="foreign-panel" style="position: fixed; bottom: 50px; right: 50px; display: none; background: white; padding: 10px; border: 1px solid black; z-index: 1000;">
        <h4>Résultats des Français de l'étranger</h4>
    """

    for idx, (circ, desc) in enumerate(foreign_circonscriptions, 1):
        foreign_html += f"""
        <p>{circ} - {desc}</p>
        <select id="foreign_party_{idx}" class="foreign-select" onchange="updateForeignParty('{idx}')">
            <option value="Non défini">Non défini</option>
            <option value="PS - Front populaire">PS - Front populaire</option>
            <option value="LFI - Front populaire">LFI - Front populaire</option>
            <option value="EELV - Front populaire">EELV - Front populaire</option>
            <option value="PCF - Front populaire">PCF - Front populaire</option>
            <option value="Autres - Front populaire">Autres - Front populaire</option>
            <option value="Modem - Ensemble">Modem - Ensemble</option>
            <option value="Renaissance - Ensemble">Renaissance - Ensemble</option>
            <option value="Autre - Ensemble">Autre - Ensemble</option>
            <option value="LR">LR</option>
            <option value="LR - RN">LR - RN</option>
            <option value="RN">RN</option>
            <option value="Reconquête">Reconquête</option>
            <option value="Independant">Independant</option>
        </select>
        <input type="hidden" id="foreign_initial_party_{idx}" value="Non défini">
        """

    foreign_html += """
    </div>
    """

    m.get_root().html.add_child(folium.Element(foreign_html))

    # Ajouter le script pour gérer le panneau repliable
    script_toggle = """
    <script>
    function toggleForeignPanel() {
        var panel = document.getElementById('foreign-panel');
        var button = document.getElementById('toggle-foreign-panel');
        if (panel.style.display === 'none' || panel.style.display === '') {
            panel.style.display = 'block';
            button.innerText = 'Réduire les résultats des Français de l\'étranger';
        } else {
            panel.style.display = 'none';
            button.innerText = 'Afficher les résultats des Français de l\'étranger';
        }
    }
    </script>
    """
    m.get_root().html.add_child(folium.Element(script_toggle))

    # Enregistrer la carte dans un fichier HTML
    m.save('circonscriptions_interactive_map26.html')

    print("La carte a été créée avec succès et enregistrée sous 'circonscriptions_interactive_map.html'.")
else:
    print("Aucun fichier sélectionné.")
 


2024-06-16 16:41:11.896 Python[25348:945377] +[CATransaction synchronize] called within transaction
2024-06-16 16:41:11.962 Python[25348:945377] +[CATransaction synchronize] called within transaction
2024-06-16 16:41:12.084 Python[25348:945377] +[CATransaction synchronize] called within transaction
2024-06-16 16:41:13.838 Python[25348:945377] +[CATransaction synchronize] called within transaction


OSError: [Errno 30] Read-only file system: 'circonscriptions_interactive_map26.html'